# Determine mooring system configuration and establish mean forces

In [1]:
import numpy as np
import pandas as pd
import math as math
from scipy import optimize
import matplotlib.pylab as plt
from matplotlib.legend_handler import HandlerLine2D

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
pass

In [2]:
# Mooring system calculation
# Define properties of the lines
# Keeping all variables in SI Units (kg, m, s) and force in N
#
g = 9.81 #m/s2

# Set up linespec
# 
linespec = pd.DataFrame(columns=['ltype','nlsgm','lsegm','unitw','ea','length'])
#
# fill in values for the lines, requires number of types and number of segments
#
#
#
nlinetypes = 1
nlinesegm = (1,3)

linespec.at[1,"ltype"] = 1
linespec.at[1,"nlsgm"] = 3
linespec.at[1,"lsegm"] = 1
linespec.at[1,"unitw"] = 376.0
linespec.at[1,"ea"] = 1725000000.
linespec.at[1,"length"] = 600.0
linespec.at[2,"ltype"] = 1
linespec.at[2,"nlsgm"] = 3
linespec.at[2,"lsegm"] = 2
linespec.at[2,"unitw"] = 10.9
linespec.at[2,"ea"] = 242000000.
linespec.at[2,"length"] = 1225.0
linespec.at[3,"ltype"] = 1
linespec.at[3,"nlsgm"] = 3
linespec.at[3,"lsegm"] = 3
linespec.at[3,"unitw"] = 376.0
linespec.at[3,"ea"] = 1725000000.
linespec.at[3,"length"] = 60.0

#
# fill in values for the mooring, requires number of lines and pretensions (pretensions not relevant in this version)
# Units in kg, m, N, degrees.
#
moorspec = pd.DataFrame(columns=['lfl','angfl','la','anga','ltype','pret'])
nlines = 12
moorspec.at[1,"lfl"] = 36.
moorspec.at[1,"angfl"] = 40.
moorspec.at[1,"la"] = 1585.94
moorspec.at[1,"anga"] = 40.
moorspec.at[1,"ltype"] = 1
moorspec.at[1,"pret"] = 1000000.
moorspec.at[2,"lfl"] = 36.
moorspec.at[2,"angfl"] = 45.
moorspec.at[2,"la"] = 1585.94
moorspec.at[2,"anga"] = 45.
moorspec.at[2,"ltype"] = 1
moorspec.at[2,"pret"] = 1000000.
moorspec.at[3,"lfl"] = 36.
moorspec.at[3,"angfl"] = 50.
moorspec.at[3,"la"] = 1585.94
moorspec.at[3,"anga"] = 50.
moorspec.at[3,"ltype"] = 1
moorspec.at[3,"pret"] = 1000000.
moorspec.at[4,"lfl"] = 36.
moorspec.at[4,"angfl"] = 130.
moorspec.at[4,"la"] = 1585.94
moorspec.at[4,"anga"] = 130.
moorspec.at[4,"ltype"] = 1
moorspec.at[4,"pret"] = 1000000.
moorspec.at[5,"lfl"] = 36.
moorspec.at[5,"angfl"] = 135.
moorspec.at[5,"la"] = 1585.94
moorspec.at[5,"anga"] = 135.
moorspec.at[5,"ltype"] = 1
moorspec.at[5,"pret"] = 1000000.
moorspec.at[6,"lfl"] = 36.
moorspec.at[6,"angfl"] = 140.
moorspec.at[6,"la"] = 1585.94
moorspec.at[6,"anga"] = 140.
moorspec.at[6,"ltype"] = 1
moorspec.at[6,"pret"] = 1000000.
moorspec.at[7,"lfl"] = 36.
moorspec.at[7,"angfl"] = 220.
moorspec.at[7,"la"] = 1585.94
moorspec.at[7,"anga"] = 220.
moorspec.at[7,"ltype"] = 1
moorspec.at[7,"pret"] = 1000000.
moorspec.at[8,"lfl"] = 36.
moorspec.at[8,"angfl"] = 225.
moorspec.at[8,"la"] = 1585.94
moorspec.at[8,"anga"] = 225.
moorspec.at[8,"ltype"] = 1
moorspec.at[8,"pret"] = 1000000.
moorspec.at[9,"lfl"] = 36.
moorspec.at[9,"angfl"] = 230.
moorspec.at[9,"la"] = 1585.94
moorspec.at[9,"anga"] = 230.
moorspec.at[9,"ltype"] = 1
moorspec.at[9,"pret"] = 1000000.
moorspec.at[10,"lfl"] = 36.
moorspec.at[10,"angfl"] = 310.
moorspec.at[10,"la"] = 1585.94
moorspec.at[10,"anga"] = 310.
moorspec.at[10,"ltype"] = 1
moorspec.at[10,"pret"] = 1000000.
moorspec.at[11,"lfl"] = 36.
moorspec.at[11,"angfl"] = 315.
moorspec.at[11,"la"] = 1585.94
moorspec.at[11,"anga"] = 315.
moorspec.at[11,"ltype"] = 1
moorspec.at[11,"pret"] = 1000000.
moorspec.at[12,"lfl"] = 36.
moorspec.at[12,"angfl"] = 320.
moorspec.at[12,"la"] = 1585.94
moorspec.at[12,"anga"] = 320.
moorspec.at[12,"ltype"] = 1
moorspec.at[12,"pret"] = 1000000.

#
#
d0 = 820 #m fairlead height above seabed

#
picon = np.pi/180.

# Define all functions
# We carry forward the functions from the previous notebook.

def catenary_end_xzle(lend,jline, V0, H0, L_seab):
    Lchb = L_seab
    xtot = 0.
    ztot = 0.
    V1 = V0
#    print("H0,L_pm,Lchb,W_pm,EA_pm")
#    print(nsegm)
    for n in range(0,nsegm):
        npos = jline + n
#        print(npos)
        W_pm = linespec.at[npos,"unitw"]
        EA_pm = linespec.at[npos,"ea"]
        L_pm = linespec.at[npos,"length"]
        if n == (nsegm-1):
            if lend > 0.:
                L_pm = lend
#  The end of a segment is changed to a different length
#        print("H0,L_seab,n,V1,Lchb,xtot,ztot")
        if Lchb >= L_pm: 
#            print()
            if H0 < 0.0:
                dx = -(abs(H0)/EA_pm + 1) * L_pm
            else:
                dx = (H0/EA_pm + 1) * L_pm
            Lchb = Lchb - abs(dx) 
            V1 = V0
            dz = 0.
        else:
            L = L_pm - Lchb
            W = W_pm*L*g
            V1 = V0 - W
#            print(H0,L_pm,Lchb,W_pm,EA_pm)
            z_t1 = 1 + (V0/H0)**2
            z_t2 = 1 + (V1/H0)**2
            dz = W*L/EA_pm * (1/2-V0/W) - abs(H0)*L/W*(np.sqrt(z_t1) - np.sqrt(z_t2))
            x_susp = abs(H0)*L/EA_pm + abs(H0)*L/W*(np.arcsinh(V0/abs(H0)) - np.arcsinh(V1/abs(H0)))
            if H0 < 0.0:
                dx = -Lchb * (1 + abs(H0)/EA_pm) - x_susp
            else:
                dx = Lchb * (1 + H0/EA_pm) + x_susp
            Lchb = 0.
            V0 = V1
        xtot = xtot + dx
        ztot = ztot + dz
#        print(H0, L_seab,n,V1,Lchb,xtot,ztot)        
    tens = np.sqrt(V1**2 + H0**2) 
#    print()
    return xtot, ztot, V1, tens
#
#
def catenary_xz(W_pm, L_tot, EA, V0, H0, L_susp, ds):
        
    W = W_pm * L_susp * g
    
    # c_s = [0]
    # c_x = [L_tot - L_susp]
    # c_z = [0]
    # c_H = [H0]
    # c_V = [V0]
    
    c_s = [0, 0]    
    c_x = [0, L_tot - L_susp]
    c_z = [0, 0]
    c_H = [H0, H0]
    c_V = [V0, V0]
    
    Ns = math.floor(L_susp/ds)
    ds = L_susp/Ns
    for i in range(1,Ns+1):
        s = c_s[-1]+ds        
        z_t1 = 1 + (V0/H0)**2
        z_t2 = 1 + ((V0 - W*s/L_susp)/H0)**2
        z = W*s/EA * (s/2/L_susp-V0/W) - H0*L_susp/W*(np.sqrt(z_t1) - np.sqrt(z_t2))        
        x_susp = H0*s/EA + H0*L_susp/W*(np.arcsinh(V0/H0) - np.arcsinh((V0 - W*s/L_susp)/H0))        
        V = V0 + W_pm*s*g
        x = L_tot - L_susp + x_susp
        
        c_s.append(s)        
        c_x.append( x )
        c_z.append( z )        
        c_V.append( V )
        
    print(c_s[-1], c_x[-1], c_z[-1], c_V[-1])
        
    return c_s, c_x, c_z, c_V
#
#
def objective_fnc(sol, *data):        
    H0 = sol[0]
    L_seab = sol[1] #Length on Seabed
    lend, jline, V0, Ltotal, xtarg, ztarg  = data
    xtot, ztot, vert, tens = catenary_end_xzle(lend,jline, V0, H0, L_seab)
    return np.array( [abs(ztarg-ztot),  abs(xtarg-xtot)] )
# Initial position
# For the given problem, both lines have the same properties.
# The target position is specified by x0,z0, indicating the centre of the body.
# 
fresh = []
fresv = []
xfl = []
yfl = []
xa = []
ya = []
lm = []
gamma = []
for i in range(1,nlines+1):
    xfl.append(moorspec.at[i,"lfl"] * np.cos(picon*moorspec.at[i,"angfl"]))
    yfl.append(moorspec.at[i,"lfl"] * np.sin(picon*moorspec.at[i,"angfl"]))
    xa.append(moorspec.at[i,"la"] * np.cos(picon*moorspec.at[i,"anga"]))
    ya.append(moorspec.at[i,"la"] * np.sin(picon*moorspec.at[i,"anga"]))
    ico = i - 1
    xd = xa[ico] - xfl[ico]
    yd = ya[ico] - yfl[ico]
    lm.append(np.sqrt(xd**2 + yd**2))
    gamma.append(np.arctan2(yd,xd) / picon)
    
# 
# Calculate horizontal force for line configuration
# 
frestotx = 0.
frestoty = 0.
frestotz = 0.
lstart = 50.0
hstart = 100000.0
print("Forces per line:")
for i in range(1,nlines+1):
    ico = i - 1
    j = 1
    mtype = moorspec.at[i,"ltype"]
    ltype = linespec.at[j,"ltype"]
    while ltype != mtype:
        j += 1
        ltype = linespec.at[j,"ltype"]
    jline = j
    nsegm = linespec.at[jline,"nlsgm"]
    Ltotal = 0.
    for n in range(0,nsegm):
        npos = jline + n
        L_pm = linespec.at[npos,"length"]
        Ltotal = Ltotal + L_pm
    V0 = 0.
    lend = 0.
    lstart = 50.0
    hstart = 100000.0
    x1_targ, z1_targ = (lm[i-1],d0)
    data = (lend,jline,V0,Ltotal,x1_targ, z1_targ)
    H0, L_seab = optimize.fsolve(objective_fnc,np.array([hstart,lstart]),args=data)
# Calculate line to obtain vertical force
#
    xval, zval, vert, tens = catenary_end_xzle(lend,jline, V0, H0, L_seab)
    lstart = L_seab
    hstart = H0
    fresh.append(H0)
    fresv.append(vert)

    print("Line ",i,"Total Length ",Ltotal,"Hor ",f"{H0:.3}","Vert ",f"{vert:.3}","Tens ",f"{tens:.3}","Line on seabed ",f"{L_seab:.5}","X ",f"{xval:.5}","Z ",f"{zval:.5}")

#
#  Calculate total force on system:
#
    gline = picon * gamma[i-1]
    frestotx = frestotx + H0 * np.cos(gline)
    frestoty = frestoty + H0 * np.sin(gline)
    frestotz = frestotz + vert
print("Mean Forces for system determined (no tension optimisation):")
print("Forces systen ","Fx ",f"{frestotx:.4}","Fy ",f"{frestoty:.4}","Fz ",f"{frestotz:.4}")
# print(fresh,fresv)
#


Forces per line:
Line  1 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  2 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  3 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  4 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  5 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  6 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  7 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  8 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  9 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e

Establish Tension level
This is done by providing tensions for various end lengths of the lines

In [3]:
# Mooring system calculation
import numpy as np
import pandas as pd
import math as math
from scipy import optimize
import matplotlib.pylab as plt
from matplotlib.legend_handler import HandlerLine2D

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

# Define properties of the lines
# Keeping all variables in SI Units (kg, m, s) and force in N
#
g = 9.81 #m/s2
#d0 = 200 #m still-water depth
#
picon = np.pi/180.

# Define all functions
# We carry forward the functions from the previous notebook.

#
# Initial position
# For the given problem, both lines have the same properties.
# The target position is specified by x0,z0, indicating the centre of the body.
# 
fresh = []
fresv = []
xfl = []
yfl = []
xa = []
ya = []
lm = []
gamma = []
for i in range(1,nlines+1):
    xfl.append(moorspec.at[i,"lfl"] * np.cos(picon*moorspec.at[i,"angfl"]))
    yfl.append(moorspec.at[i,"lfl"] * np.sin(picon*moorspec.at[i,"angfl"]))
    xa.append(moorspec.at[i,"la"] * np.cos(picon*moorspec.at[i,"anga"]))
    ya.append(moorspec.at[i,"la"] * np.sin(picon*moorspec.at[i,"anga"]))
    ico = i - 1
    xd = xa[ico] - xfl[ico]
    yd = ya[ico] - yfl[ico]
    lm.append(np.sqrt(xd**2 + yd**2))
    gamma.append(np.arctan2(yd,xd) / picon)
    
# 
# Calculate horizontal force for line configuration
# 
frestotx = 0.
frestoty = 0.
frestotz = 0.
lstart = 50.0
hstart = 100000.0
for i in range(1,nlines+1):
    ico = i - 1
    j = 1
    mtype = moorspec.at[i,"ltype"]
    ltype = linespec.at[j,"ltype"]
    while ltype != mtype:     
        j += 1
        ltype = linespec.at[j,"ltype"]
    jline = j
    nsegm = linespec.at[jline,"nlsgm"]
    npos = jline + nsegm - 1
    L_pm = linespec.at[npos,"length"]
    kstep = 10
    lend = 0.
    lstep = L_pm / kstep
    for k in range(1,kstep+1):
        V0 = 0.
        lend = lend + lstep
        print(k,L_pm,lend)
        x1_targ, z1_targ = (lm[i-1],d0)
        data = (lend,jline,V0,Ltotal,x1_targ, z1_targ)
        H0, L_seab = optimize.fsolve(objective_fnc,np.array([hstart,lstart]),args=data)
        xval, zval, vert, tens = catenary_end_xzle(lend,jline, V0, H0, L_seab)
        print("Tension at line ",i,"Tension [kN] ",f"{(tens/1000.):.3}","Length last segment: ",f"{lend:.4}","Original length ",f"{L_pm:.4}")
        lstart = L_seab
        hstart = H0
    fresh.append(H0)
    fresv.append(vert)
    print()
#    print("Forces Line ",i,"Hor ",f"{H0:.3}","Vert ",f"{vert:.3}","Tens ",f"{tens:.3}","Line on seabed ",f"{L_seab:.2}","X ",f"{xval:.4}","Z ",f"{zval:.4}")
    print()
#
#  Calculate total force on system:
#
    gline = picon * gamma[i-1]
    frestotx = frestotx + H0 * np.cos(gline)
    frestoty = frestoty + H0 * np.sin(gline)
    frestotz = frestotz + vert
print()
print("Forces systen ","Fx ",f"{frestotx:.4}","Fy ",f"{frestoty:.4}","Fz ",f"{frestotz:.4}")
# print(fresh,fresv)
#


1 60.0 6.0
Tension at line  1 Tension [kN]  9.31e+02 Length last segment:  6.0 Original length  60.0
2 60.0 12.0
Tension at line  1 Tension [kN]  7.66e+02 Length last segment:  12.0 Original length  60.0
3 60.0 18.0
Tension at line  1 Tension [kN]  6.22e+02 Length last segment:  18.0 Original length  60.0
4 60.0 24.0
Tension at line  1 Tension [kN]  5.1e+02 Length last segment:  24.0 Original length  60.0
5 60.0 30.0
Tension at line  1 Tension [kN]  4.37e+02 Length last segment:  30.0 Original length  60.0
6 60.0 36.0
Tension at line  1 Tension [kN]  4e+02 Length last segment:  36.0 Original length  60.0
7 60.0 42.0
Tension at line  1 Tension [kN]  3.85e+02 Length last segment:  42.0 Original length  60.0
8 60.0 48.0
Tension at line  1 Tension [kN]  3.83e+02 Length last segment:  48.0 Original length  60.0
9 60.0 54.0
Tension at line  1 Tension [kN]  3.88e+02 Length last segment:  54.0 Original length  60.0
10 60.0 60.0
Tension at line  1 Tension [kN]  3.98e+02 Length last segment:  60

Recalculate average position based on an update of the last segment lengths / pretensions

In [4]:
# Mooring system calculation
import numpy as np
import pandas as pd
import math as math
from scipy import optimize
import matplotlib.pylab as plt
from matplotlib.legend_handler import HandlerLine2D

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
pass
# Define properties of the lines
# Keeping all variables in SI Units (kg, m, s) and force in N
#
g = 9.81 #m/s2

# Set up linespec
# 
linespec = pd.DataFrame(columns=['ltype','nlsgm','lsegm','unitw','ea','length'])
#
# update last line segment if needed 
#
#
nlinetypes = 1
nlinesegm = (1,3)

linespec.at[1,"ltype"] = 1
linespec.at[1,"nlsgm"] = 3
linespec.at[1,"lsegm"] = 1
linespec.at[1,"unitw"] = 376.0
linespec.at[1,"ea"] = 1725000000.
linespec.at[1,"length"] = 600.0
linespec.at[2,"ltype"] = 1
linespec.at[2,"nlsgm"] = 3
linespec.at[2,"lsegm"] = 2
linespec.at[2,"unitw"] = 10.9
linespec.at[2,"ea"] = 242000000.
linespec.at[2,"length"] = 1225.0
linespec.at[3,"ltype"] = 1
linespec.at[3,"nlsgm"] = 3
linespec.at[3,"lsegm"] = 3
linespec.at[3,"unitw"] = 376.0
linespec.at[3,"ea"] = 1725000000.
linespec.at[3,"length"] = 60.0

# Recalculate based on updated tension level. 
# Initial position
# For the given problem, both lines have the same properties.
# The target position is specified by x0,z0, indicating the centre of the body.
# 
fresh = []
fresv = []
xfl = []
yfl = []
xa = []
ya = []
lm = []
gamma = []
for i in range(1,nlines+1):
    xfl.append(moorspec.at[i,"lfl"] * np.cos(picon*moorspec.at[i,"angfl"]))
    yfl.append(moorspec.at[i,"lfl"] * np.sin(picon*moorspec.at[i,"angfl"]))
    xa.append(moorspec.at[i,"la"] * np.cos(picon*moorspec.at[i,"anga"]))
    ya.append(moorspec.at[i,"la"] * np.sin(picon*moorspec.at[i,"anga"]))
    ico = i - 1
    xd = xa[ico] - xfl[ico]
    yd = ya[ico] - yfl[ico]
    lm.append(np.sqrt(xd**2 + yd**2))
    gamma.append(np.arctan2(yd,xd) / picon)
    
# 
# Calculate horizontal force for line configuration
# 
frestotx = 0.
frestoty = 0.
frestotz = 0.
lstart = 50.0
hstart = 100000.0
print("Forces per line:")
for i in range(1,nlines+1):
    ico = i - 1
    j = 1
    mtype = moorspec.at[i,"ltype"]
    ltype = linespec.at[j,"ltype"]
    while ltype != mtype:
        j += 1
        ltype = linespec.at[j,"ltype"]
#    print(i,mtype,ltype,j)
    jline = j
    nsegm = linespec.at[jline,"nlsgm"]
    Ltotal = 0.
    for n in range(0,nsegm):
        npos = jline + n
        L_pm = linespec.at[npos,"length"]
        Ltotal = Ltotal + L_pm
    V0 = 0.
    lend = 0.
    lstart = 50.0
    hstart = 100000.0
    x1_targ, z1_targ = (lm[i-1],d0)
    data = (lend,jline,V0,Ltotal,x1_targ, z1_targ)
    H0, L_seab = optimize.fsolve(objective_fnc,np.array([hstart,lstart]),args=data)
# Calculate line to obtain vertical force
#
    xval, zval, vert, tens = catenary_end_xzle(lend,jline, V0, H0, L_seab)

    fresh.append(H0)
    fresv.append(vert)
    hstart = H0
    lstart = L_seab

    print("Line ",i,"Total Length ",Ltotal,"Hor ",f"{H0:.3}","Vert ",f"{vert:.3}","Tens ",f"{tens:.3}","Line on seabed ",f"{L_seab:.5}","X ",f"{xval:.5}","Z ",f"{zval:.4}")

#
#  Calculate total force on system:
#
    gline = picon * gamma[i-1]
    frestotx = frestotx + H0 * np.cos(gline)
    frestoty = frestoty + H0 * np.sin(gline)
    frestotz = frestotz + vert
print("Mean Forces for system determined (with tension optimisation):")
print("Forces systen ","Fx ",f"{frestotx:.4}","Fy ",f"{frestoty:.4}","Fz ",f"{frestotz:.4}")
# print(fresh,fresv)
#


Forces per line:
Line  1 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  2 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  3 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  4 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  5 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  6 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  7 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  8 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e+05 Tens  3.98e+05 Line on seabed  591.65 X  1549.9 Z  820.0
Line  9 Total Length  1885.0 Hor  1.09e+05 Vert  -3.83e

Analysis of a Force - Excursion diagram based on an excursion path

In [5]:
# Mooring system calculation
import numpy as np
import pandas as pd
import math as math
from scipy import optimize
import matplotlib.pylab as plt
from matplotlib.legend_handler import HandlerLine2D

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 5


# 
fresxt = []
fresyt = []
freszt = []
xfl = []
yfl = []
xa = []
ya = []
lm = []
gamma = []
for i in range(1,nlines+1):
    xfl.append(moorspec.at[i,"lfl"] * np.cos(picon*moorspec.at[i,"angfl"]))
    yfl.append(moorspec.at[i,"lfl"] * np.sin(picon*moorspec.at[i,"angfl"]))
    xa.append(moorspec.at[i,"la"] * np.cos(picon*moorspec.at[i,"anga"]))
    ya.append(moorspec.at[i,"la"] * np.sin(picon*moorspec.at[i,"anga"]))
    ico = i - 1
    xd = xa[ico] - xfl[ico]
    yd = ya[ico] - yfl[ico]
    lm.append(np.sqrt(xd**2 + yd**2))
    gamma.append(np.arctan2(yd,xd) / picon)
    
# 
# Create path for excursion
# 
lstart = 50.0
hstart = 100000.0
deltastep = 1.0 # m
deltaangle = 45.0 # degrees
path = []
kmax = 31
for k in range(0,kmax):
#for k in range(1,3):
    delta = k * deltastep
    path.append(delta)
# 
# Calculate horizontal force for line configuration
# 
    frestotx = 0.
    frestoty = 0.
    frestotz = 0.
    for i in range(1,nlines+1):
        ico = i - 1
        j = 1
        mtype = moorspec.at[i,"ltype"]
        ltype = linespec.at[j,"ltype"]
        while ltype != mtype:
            j += 1
            ltype = linespec.at[j,"ltype"]
        jline = j
        nsegm = linespec.at[jline,"nlsgm"]
        Ltotal = 0.
        for n in range(0,nsegm):
            npos = jline + n
            L_pm = linespec.at[npos,"length"]
            Ltotal = Ltotal + L_pm
        V0 = 0.
        lend = 0.
        nflx = delta * np.cos(picon*deltaangle) + xfl[ico]
        nfly = delta * np.sin(picon*deltaangle) + yfl[ico]
        nxd = xa[ico] - nflx
        nyd = ya[ico] - nfly
        nlm = np.sqrt(nxd**2 + nyd**2)
        ndel = np.arctan2(nyd,nxd) / picon
# Note that the intial estimates make use of the calculation of the previous mean position calculation.
# that could create slight difference with previous mean position calculation.
        x1_targ, z1_targ = (nlm,d0)
        data = (lend,jline,V0,Ltotal,x1_targ, z1_targ)
        H0, L_seab = optimize.fsolve(objective_fnc,np.array([hstart,lstart]),args=data)
# Recalculate line to obtain vertical force
#
        xval, zval, vert, tens = catenary_end_xzle(lend,jline, V0, H0, L_seab)

        hstart = H0
        lstart = L_seab
#
#  Calculate total force on system:
#
        gline = picon * gamma[ico]
        frestotx = frestotx + H0 * np.cos(gline)
        frestoty = frestoty + H0 * np.sin(gline)
        frestotz = frestotz + vert

    print("Forces for system determined with step ",delta, " at angle ",deltaangle)
    print("Forces systen ","Fx ",f"{frestotx:.4}","Fy ",f"{frestoty:.4}","Fz ",f"{frestotz:.4}")        
    fresxt.append(frestotx)
    fresyt.append(frestoty)
    freszt.append(frestotz)
print()
print(path,fresxt,fresyt,freszt)
#


Forces for system determined with step  0.0  at angle  45.0
Forces systen  Fx  -3.958e-09 Fy  -3.216e-09 Fz  -4.597e+06
Forces for system determined with step  1.0  at angle  45.0
Forces systen  Fx  -6.524e+03 Fy  -6.524e+03 Fz  -4.597e+06
Forces for system determined with step  2.0  at angle  45.0
Forces systen  Fx  -1.306e+04 Fy  -1.306e+04 Fz  -4.597e+06
Forces for system determined with step  3.0  at angle  45.0
Forces systen  Fx  -1.961e+04 Fy  -1.961e+04 Fz  -4.598e+06
Forces for system determined with step  4.0  at angle  45.0
Forces systen  Fx  -2.62e+04 Fy  -2.62e+04 Fz  -4.599e+06
Forces for system determined with step  5.0  at angle  45.0
Forces systen  Fx  -3.283e+04 Fy  -3.283e+04 Fz  -4.6e+06
Forces for system determined with step  6.0  at angle  45.0
Forces systen  Fx  -3.952e+04 Fy  -3.952e+04 Fz  -4.601e+06
Forces for system determined with step  7.0  at angle  45.0
Forces systen  Fx  -4.627e+04 Fy  -4.627e+04 Fz  -4.603e+06
Forces for system determined with step  8.0 